In [2]:
## init object_explorer

from datetime import datetime
from uuid import uuid4
import hmac
import hashlib
import base64
import jaydebeapi
from dotenv import dotenv_values
import polars as pl
env = dotenv_values("../.env")

query = """
   SELECT table_qualifier,
          table_name,
          remarks
     FROM oa_tables AS T
"""
schema = { "table_qualifier": pl.Utf8, "table_name": pl.Utf8, "remarks": pl.Utf8 }

# generate password
timestamp = str(int(datetime.now().timestamp()))
nonce = uuid4().hex

base_string = "&".join((env["ACCOUNTID"], env["CONSUMERKEY"], env["TOKENID"], nonce, timestamp))
signature_key = "&".join((env["CONSUMERSECRET"], env["TOKENSECRET"]))

signature = base64.b64encode(hmac.new(
        signature_key.encode(),
        base_string.encode(),
        hashlib.sha256).digest()
    ).decode()

token_password = "&".join((base_string, signature, "HMAC-SHA256"))

# make connection
conn = jaydebeapi.connect(
    "com.netsuite.jdbc.openaccess.OpenAccessDriver",
    f'jdbc:ns://{env["ACCOUNTID"]}.connect.api.netsuite.com:1708;ServerDataSource=NetSuite2.com;encrypted=1;NegotiateSSLClose=false;tcpKeepAlive=true;CustomProperties=(AccountID={env["ACCOUNTID"]};RoleID={env["ROLEID"]})',
    [env["USER"], token_password],
    "../NQjc.jar"
    )

curs = conn.cursor()
curs.execute(query)

data=curs.fetchall()
df = pl.DataFrame(data, schema=schema)
df.write_parquet(f"./object_explorer.parquet")
